In [69]:
from bokeh.plotting import figure, save
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
import geopandas as gpd
import pysal as ps
import pandas as pd



In [70]:
grid_fp = r"C:/Users/Asus/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/tract2010/tract2010.shp"
network_fp = r"C:/Users/Asus/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/trans_network_bus.shp"

grid = gpd.read_file(grid_fp)
network = gpd.read_file(network_fp)
grid = grid.loc[grid['COUNTYFP10'] == '033']


In [39]:
income_df = pd.read_csv('C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Docs/income_tract.csv')
   

In [71]:
grid['geometry'] = grid['geometry'].to_crs(crs='+init=epsg:4326')
network['geometry'] = network['geometry'].to_crs(crs='+init=epsg:4326')


In [72]:
def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]
    
def getLineCoords(geometry, coord_type):
    """ Returns Coordinates of Linestring object."""
    return getXYCoords(geometry, coord_type)

def getPolyCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

def getCoords(row, geom_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon). 
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geom_col]
    
    # Check the geometry type
    gtype = geom.geom_type
    
    # "Normal" geometries
    # -------------------
    
    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )


grid['x'] = grid.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
grid['y'] = grid.apply(getCoords, geom_col="geometry", coord_type="y", axis=1)

network['x'] = network.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
network['y'] = network.apply(getCoords, geom_col="geometry", coord_type="y", axis=1)


In [108]:
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)
n_df = network.drop('geometry', axis=1).copy()
nsource = ColumnDataSource(n_df)

In [57]:
income_df = pd.read_csv('C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Data/income_tract.csv')

In [60]:
income_df['htract'].head ()

0    1.00
1    2.00
2    3.00
3    4.01
4    4.02
Name: htract, dtype: float64

In [74]:
grid['NAME10'] = pd.to_numeric(grid['NAME10'])

In [75]:
grid = grid.merge(income_df, left_on='NAME10', right_on='htract', how='inner')

In [79]:
grid.head()

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,x,y,Unnamed: 0,htract,hh_income_detailed,income
0,53,033,022006,53033022006,220.06,Census Tract 220.06,G5020,S,1408907,0,+47.7043393,-122.1992667,"POLYGON ((-122.210483 47.70591899998356, -122....","[-122.21048300000004, -122.21052900000001, -12...","[47.70591899998356, 47.707340999983565, 47.707...",160,220.06,6.75,6
1,53,033,032320,53033032320,323.20,Census Tract 323.20,G5020,S,9554873,38494,+47.7564769,-122.1244452,"POLYGON ((-122.131819 47.77569299998373, -122....","[-122.13181900000002, -122.131994, -122.131829...","[47.77569299998373, 47.77579799998373, 47.7757...",374,323.20,6.00,5
2,53,033,031703,53033031703,317.03,Census Tract 317.03,G5020,S,7139218,643846,+47.3716888,-122.1484314,"POLYGON ((-122.165413 47.37570299998269, -122....","[-122.16541300000004, -122.16541200000003, -12...","[47.37570299998269, 47.3763489999827, 47.37699...",335,317.03,7.50,7
3,53,033,031206,53033031206,312.06,Census Tract 312.06,G5020,S,3668962,60430,+47.3135621,-122.1835380,"POLYGON ((-122.170459 47.31165399998253, -122....","[-122.17045900000002, -122.17137600000002, -12...","[47.31165399998253, 47.30995899998253, 47.3095...",326,312.06,2.00,1
4,53,033,032102,53033032102,321.02,Census Tract 321.02,G5020,S,79040166,84391,+47.4851508,-121.9657840,"POLYGON ((-122.033667 47.47679999998297, -122....","[-122.03366700000001, -122.03441200000003, -12...","[47.476799999982966, 47.47755999998296, 47.478...",353,321.02,6.20,6


In [107]:
breaks = [x for x in range(1, 11, 2)]

# Initialize the classifier and apply it
classifier = ps.User_Defined.make(bins=breaks)
pt_classif = grid[['hh_income_detailed']].apply(classifier)

# Rename the classified column
pt_classif.columns = ['income2']

# Join it back to the grid layer
grid = grid.join(pt_classif)

In [116]:
from bokeh.palettes import RdYlGn10 as palette
from bokeh.models import LogColorMapper

# Create the color mapper

color_mapper = LogColorMapper(palette=palette)

p = figure(title="PSRC Income")

# Plot grid
grid2 = p.patches('x', 'y', source=gsource,
         fill_color={'field': 'income2', 'transform': color_mapper},
         fill_alpha=1.0, line_color="black", line_width=0.05, legend="income2")

p.legend.location = "top_right"
p.legend.orientation = "vertical"

#p.multi_line('x', 'y', source=nsource, color="red", line_width=0.5)

ghover = HoverTool(renderers=[grid2])
ghover.tooltips=[("Income Category", "@income2")]
              

p.add_tools(ghover)

outfp = r"C:/Users/ASUS/Desktop/uwsed/uwseds-group-transit-and-social-science/Docs/map.html"
save(p, outfp)



C:\Users\ASUS\Anaconda3\lib\site-packages\bokeh\io.py:527: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\ASUS\Anaconda3\lib\site-packages\bokeh\io.py:537: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\Users\\ASUS\\Desktop\\uwsed\\uwseds-group-transit-and-social-science\\Docs\\map.html'